#### Cleaning Data activity and analysing with categorical data - Label Encoding - One Hot Encoding

In [1]:
#Importing all necessary libraries 

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [2]:
# loading dataset

companies = pd.read_csv(r"C:\Users\PRIYA SAVIO\Downloads\Modified_Unicorn_Companies.csv")


In [3]:
# displaying the maximum columns as pandas tend to truncate the columns while displaying
# running at the start of the session will set this setting throughout

pd.set_option('display.max_columns',None)

In [4]:
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG"
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita..."


In [5]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074 entries, 0 to 1073
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Company           1074 non-null   object
 1   Valuation         1074 non-null   int64 
 2   Date Joined       1074 non-null   object
 3   Industry          1074 non-null   object
 4   City              1057 non-null   object
 5   Country/Region    1074 non-null   object
 6   Continent         1074 non-null   object
 7   Year Founded      1074 non-null   int64 
 8   Funding           1074 non-null   object
 9   Select Investors  1074 non-null   object
dtypes: int64(2), object(8)
memory usage: 84.0+ KB


In [6]:
# changing the Date Joined column to Datetime dtype

companies['Date Joined'] = pd.to_datetime(companies['Date Joined'])

In [7]:
companies['Date Joined'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1074 entries, 0 to 1073
Series name: Date Joined
Non-Null Count  Dtype         
--------------  -----         
1074 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 8.5 KB


Add a column called `Years To Unicorn`, which is the number of years between when the company was founded and when it became a unicorn.

In [8]:
companies['Years To Unicorn'] = companies['Date Joined'].dt.year - companies['Year Founded']

In [9]:
companies['Years To Unicorn'].describe()

count    1074.000000
mean        7.013035
std         5.331842
min        -3.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        98.000000
Name: Years To Unicorn, dtype: float64

#### upon checking the Years to Unicorn column with describe() we could see a value of -3 which seems to be invalid as company cannot attain unicorn before they even founded. we are isolating the negative data to explore more

In [10]:
companies[companies['Years To Unicorn']<0]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2020,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


##### We have isolated the rows which has invalid Years To Unicorn, searchin on the internet confirms that the company Invision was founded on 2011 and not 2020

In [11]:
# replacing the Year Founded with 2011 and updating the companies data

In [12]:
mask = companies['Company']=='InVision'

companies[mask]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2020,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


In [13]:
#updating the year

companies.loc[mask,'Year Founded'] = 2011

In [14]:
# now we are runing the 'Years To unicorn' step to check if the years are updated

companies['Years To Unicorn'] = companies['Date Joined'].dt.year - companies['Year Founded']

In [15]:
companies['Years To Unicorn'].describe()

count    1074.000000
mean        7.021415
std         5.323155
min         0.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        98.000000
Name: Years To Unicorn, dtype: float64

#### data corrected

#### Now checking the Industry type. Checking the entries are consistent throughout the df. All data should be even cased for example Automobile and automobile both are not the same due to case variance also needs to check the spelling and shortforms 

In [16]:
companies['Industry'].unique()

array(['Artificial intelligence', 'Other',
       'E-commerce & direct-to-consumer', 'FinTech', 'Fintech',
       'Internet software & services',
       'Supply chain, logistics, & delivery', 'Consumer & retail',
       'Data management and analytics', 'Edtech', 'Health', 'Hardware',
       'Auto & transportation', 'Travel', 'Cybersecurity',
       'Mobile & telecommunications', 'Data management & analytics',
       'Artificial Intelligence'], dtype=object)

Upon Checking the companies['Industry'] column we found there are inconsistency which created 2 or more entries for same industry type

In [17]:
# we are creating a list of correct industry type to compare

industry_list = ['Artificial intelligence', 'Other','E-commerce & direct-to-consumer', 'Fintech',
       'Internet software & services','Supply chain, logistics, & delivery', 'Consumer & retail',
       'Data management & analytics', 'Edtech', 'Health', 'Hardware','Auto & transportation',
        'Travel', 'Cybersecurity','Mobile & telecommunications']

In [18]:
difference_industry_list = set(companies['Industry']) - set(industry_list)

In [19]:
difference_industry_list

{'Artificial Intelligence', 'Data management and analytics', 'FinTech'}

There are three industry names which are inconsistent format invalid case or spelling
The correct label should be 'Artificial intelligence' , 'Data management & analytics', 'Fintech'

In [20]:
# creating replacement dictionary with the correct values

replacement_dict = {'Artificial Intelligence': 'Artificial intelligence' , 'Data management and analytics':'Data management & analytics', 
                    'FinTech':'Fintech'}

# replacing the incorrect values using the dictionary

companies['Industry'] = companies['Industry'].replace(replacement_dict)

In [21]:
companies['Industry'].unique()

array(['Artificial intelligence', 'Other',
       'E-commerce & direct-to-consumer', 'Fintech',
       'Internet software & services',
       'Supply chain, logistics, & delivery', 'Consumer & retail',
       'Data management & analytics', 'Edtech', 'Health', 'Hardware',
       'Auto & transportation', 'Travel', 'Cybersecurity',
       'Mobile & telecommunications'], dtype=object)

Confirmation on the replaced industry names

#### Check for duplicate company names in the dataset to identify potential data entry errors or repeated records.

In [22]:
companies[companies['Company'].duplicated(keep=False)]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
385,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,United Kingdom,Europe,2007,$233M,"TSG Consumer Partners, Crowdcube",10
386,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,UnitedKingdom,Europe,2007,$233M,TSG Consumer Partners,10
510,ZocDoc,2,2015-08-20,Health,New York,United States,North America,2007,$374M,"Founders Fund, Khosla Ventures, Goldman Sachs",8
511,ZocDoc,2,2015-08-20,Health,NaN,United States,North America,2007,$374M,Founders Fund,8
1031,SoundHound,1,2018-05-03,Artificial intelligence,Santa Clara,United States,North America,2005,$215M,"Tencent Holdings, Walden Venture Capital, Glob...",13
1032,SoundHound,1,2018-05-03,Other,Santa Clara,United States,North America,2005,$215M,Tencent Holdings,13


### Removing Duplicate Company Entries

There are three companies with duplicate rows due to inconsistencies in other columns.  
After verification, the first occurrence of each company is valid, and the remaining duplicates are removed.

In [23]:
companies = companies.drop_duplicates(subset='Company',keep="first")

In [24]:
companies[companies['Company'].duplicated(keep=False)]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn


### Converting the Numerical Data to Categorical Data

Converting numerical data into categorical form helps group continuous values into meaningful segments,  
making patterns easier to spot and simplifying analysis for models that handle categories better than raw numbers.

The data in the Valuation column represents how much money (in billions, USD) each company is valued at. Use the Valuation column to create a new column called High Valuation. For each company, the value in this column should be low if the company is in the bottom 50% of company valuations and high if the company is in the top 50%.

In [25]:
companies['High Valuation'] = pd.qcut(companies['Valuation'],2,labels=['Low','High'])

In [26]:
companies['High Valuation'].head()

0    High
1    High
2    High
3    High
4    High
Name: High Valuation, dtype: category
Categories (2, object): ['Low' < 'High']

Converted continuous numerical values into categorical labels (“High” and “Low”) using quantile-based binning (qcut) to simplify analysis and enable comparisons across groups.

This is called Quantile based Binning

qcut helps you bin the data meaningfully.

But for modeling or calculations, categories often need to be converted back to numeric codes or dummy variables.

### Converting Categorical data into numerical 
Converting  the continent into numberical values 

In [27]:
# checking the continents
companies['Continent'].value_counts()

Continent
North America    586
Asia             310
Europe           143
South America     21
Oceania            8
Africa             3
Name: count, dtype: int64

In [28]:
#converting the column type to category

companies['Continent'] = companies['Continent'].astype('category')

In [29]:
companies['Continent'].info()

<class 'pandas.core.series.Series'>
Index: 1071 entries, 0 to 1073
Series name: Continent
Non-Null Count  Dtype   
--------------  -----   
1071 non-null   category
dtypes: category(1)
memory usage: 9.6 KB


In [30]:
#creating a new numeric column for the continent

companies['Continent Numeric'] = companies['Continent'].cat.codes

In [31]:
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Numeric
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,High,1
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,High,3
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,High,1
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,High,3
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,High,2


#### Creating 'Country/Region' to numeric data using cat.codes label encoding

Converts categorical values into integer codes, where each unique category is assigned a numeric label.

In [32]:
companies['Country/Region'] = companies['Country/Region'].astype('category')

companies['Country/Region Numerical'] = companies['Country/Region'].cat.codes

In [40]:
print((set(companies['Country/Region Numerical']))) #This shows there are 46 unique values 

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45}


In [41]:
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Numeric,Country/Region Numerical
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,High,1,9
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,High,3,44
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,High,1,9
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,High,3,44
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,High,2,38


#### converting the Industry type to numeric with one hot encoding

In [51]:
dummy = pd.get_dummies(companies['Industry']).astype(int)

In [52]:
companies_with_dummy = pd.concat([companies,dummy],axis=1)

In [53]:
companies_with_dummy.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Numeric,Country/Region Numerical,Artificial intelligence,Auto & transportation,Consumer & retail,Cybersecurity,Data management & analytics,E-commerce & direct-to-consumer,Edtech,Fintech,Hardware,Health,Internet software & services,Mobile & telecommunications,Other,"Supply chain, logistics, & delivery",Travel
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,High,1,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,High,3,44,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,High,1,9,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,High,3,44,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,High,2,38,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### Conclusion

For columns like `Country/Region` and `Continent`, **label encoding** worked best because it’s quick, memory-friendly, and doesn’t create extra columns.  

For columns with only a few unique values, **one-hot encoding** is useful since it avoids adding any false order between categories. But for large sets like `Industry`, it can create too many columns and make the dataset heavy.  

**Key takeaway:**  
- Use **label encoding** for large or ordinal categories.  
- Use **one-hot encoding** for small sets of nominal categories.
